# Libraries

In [1]:
import pandas as pd
from datetime import datetime

# Parameters

In [2]:
TAX_RATE = 0.275
VERBOSE = True

# Functions

In [3]:
def tax_calculator(selling_units, selling_price, average_purchase_price):
    """
    Calculate the profit and taxes based on selling units, selling price,
    and average purchase price.

    The function computes the total purchase value of the units sold at
    their average purchase price, calculates the profit by subtracting
    the purchase value from the selling value, and applies taxes only
    if the profit is positive.

    Args:
        selling_units (int or float): Number of units sold.
        selling_price (float): Selling price per unit.
        average_purchase_price (float): Average purchase price per unit.

    Returns:
        tuple: A tuple containing:
            - profit (float): The calculated profit from the sale.
            - taxes (float): The calculated taxes based on profit and a
              predefined TAX_RATE. If no profit, taxes will be 0.
    """
    # Purchase value that the selling units would have if bought at the same price
    average_purchase_value = selling_units * average_purchase_price

    # Profit: current selling value - average purchase value
    sold_value = selling_units * selling_price
    profit = round((sold_value - average_purchase_value), 2)

    # Calculate taxes only if profit is positive
    taxes = 0
    if profit > 0:
        taxes = round(profit * TAX_RATE, 2)

    return profit, taxes

def calculate_units_to_remove_from_purchase_order(purchase_order_units, units_to_sell_avg, number_of_purchase_orders, counter_order):
  """
  Calculate the number of units to sell and adjust the average units to sell
  per purchase order based on the difference between available purchase order
  units and the average units to sell.

  If the available units in the current purchase order are less than the
  average units to sell, the average is adjusted for the remaining purchase
  orders, and all units from the current order are sold. Otherwise, the
  average units are sold.

  Args:
      purchase_order_units (int or float): The number of units in the current purchase order.
      units_to_sell_avg (float): The average number of units to sell from each purchase order.
      number_of_purchase_orders (int): The total number of purchase orders.
      counter_order (int): The current purchase order index being processed.

  Returns:
      tuple: A tuple containing:
          - units_to_sell (float): The actual number of units to sell from the current order.
          - units_to_sell_avg (float): The updated average units to sell for future orders.
  """

  # Calculate the difference between purchase order units and average units to sell
  units_diff = purchase_order_units - units_to_sell_avg

  if units_diff <0:
    # If not enough units are available, adjust the average units to sell
    n = number_of_purchase_orders  - counter_order # Remaining purchase orders
    units_to_sell_avg += abs(units_diff) / n # Adjust the average
    units_to_sell = purchase_order_units # Sell all available units
  else:
    # If enough units are available, sell the average amount
    units_to_sell = units_to_sell_avg

  return units_to_sell, units_to_sell_avg

def upload_balance(df_balance, df_balance_temp, total_units_to_sell):
    """
    Update the balance of units based on the total units to sell and the temporary balance
    of purchase orders. This function adjusts the units in the original balance DataFrame
    and returns the updated balance along with the total number of units sold.

    Args:
        df_balance (DataFrame): The original balance DataFrame containing current unit counts.
        df_balance_temp (DataFrame): The temporary DataFrame of purchase orders to process.
        total_units_to_sell (int or float): The total number of units to sell from the balance.

    Returns:
        tuple: A tuple containing:
            - df_balance (DataFrame): The updated balance DataFrame after processing.
            - tot_units_sold (int or float): The total number of units sold during the update.
    """

    # Define parameters for balance update
    counter_order = 1  # Count how many purchase orders have been processed
    tot_units_sold = 0  # Keep track of how many units have been removed from balance
    number_of_purchase_orders = df_balance_temp.shape[0]  # Number of purchase orders
    units_to_sell_avg = total_units_to_sell / number_of_purchase_orders  # Average units to remove from each order

    # Update balance
    for j, row_b in df_balance_temp.iterrows():
        # Remove units from purchase orders
        units_to_sell, units_to_sell_avg = calculate_units_to_remove_from_purchase_order(
            purchase_order_units=row_b['Units'],
            units_to_sell_avg=units_to_sell_avg,
            number_of_purchase_orders=number_of_purchase_orders,
            counter_order=counter_order
        )

        # Subtract the units to sell from the balance DataFrame
        df_balance.loc[j, 'Units'] -= units_to_sell

        # Update the counter for the next order
        counter_order += 1

        # Update the total number of units sold
        tot_units_sold += units_to_sell

    # Remove from balance orders which no longer have units
    df_balance = df_balance[df_balance['Units'] > 0].reset_index(drop=True)

    return df_balance, tot_units_sold

# Load data

## Purchase

In [4]:
# Data provided by the user
data = {
    "Date": ["01/09/2023", "02/10/2023", "03/11/2023", "02/12/2023", "02/01/2024", "01/02/2024",
             "01/03/2024", "02/04/2024", "02/05/2024", "03/06/2024", "01/07/2024", "01/08/2024",
             "01/09/2024", "15/09/2024", "15/10/2024"],
    "Price": [78.85, 77.04, 75.60, 79.79, 82.15, 84.99, 88.59, 90.63, 88.87, 91.82, 94.32, 95.27, 95.55, 94.70, 99.76],
    "Units": [3.152, 3.225, 13.207, 12.514, 12.154, 11.749, 11.271, 11.017, 11.236, 10.875, 10.586, 10.481, 10.450, 10.544, 10.009]
}

# Creating DataFrame
df_purchase = pd.DataFrame(data)
df_purchase['Date'] = pd.to_datetime(df_purchase['Date'], format='%d/%m/%Y')

# Sell

In [5]:
df_sales_orig = pd.DataFrame({'Date':[datetime(2024,10,16), datetime(2024,10,17)],
                         'Units': [100, 52],
                         "Price": [120, 100]})
df_sales_orig['Value'] = df_sales_orig['Units']*df_sales_orig['Price']

# Calculate Taxes

In [6]:
# Create a copy of:
df_balance = df_purchase.copy() # purchase to keep track of the balance
df_sales = df_sales_orig.copy() # sakes to enrich it with profit/tax infos

if VERBOSE:
  print("Original Balance:")
  print(df_balance,"\n")

# Loop through all sales
sales_infos = []
for i, row_s in df_sales.iterrows():

  # Select purchases until sale date
  df_balance_temp = df_balance[df_balance['Date']<=df_sales['Date'].iloc[-1]].sort_values("Units")

  # Check if units can be sold:
  if row_s['Units'] <= df_balance_temp['Units'].sum():

    # Calculate average purchasing price
    average_purchase_price = (df_balance_temp['Units']*df_balance_temp['Price']).sum()/df_balance_temp['Units'].sum()

    # Calculate profit and taxes
    profit, taxes = tax_calculator(selling_units=row_s['Units'],
                                   selling_price=row_s['Price'],
                                   average_purchase_price=average_purchase_price)

    # Store profit/taxes infos in sales dataframe
    sales_infos.append({'Gross Profit':profit, 'Taxes':taxes, 'Net Profit':profit - taxes, 'Average Purchase Price':round(average_purchase_price,2)})

    if VERBOSE:
      print(f"Average Purchase Price: {average_purchase_price}")
      print(f"Sold {row_s['Units']} units for a value of {row_s['Value']}. The {row_s['Units']} units where purchased at averaged price of {round(average_purchase_price,2)}, which means their average value is {round(row_s['Units']*average_purchase_price, 2)}")
      print(f"Gross Profit: {round(profit,2)} ({round(row_s['Value'], 2)} - {round(row_s['Units']*average_purchase_price,2)})")
      print(f"Taxes: {taxes}\n")

    # Update balance
    df_balance, tot_units_sold = upload_balance(df_balance, df_balance_temp, row_s['Units'])

    # Check if sold units match target:
    if round(row_s['Units']) != round(tot_units_sold):
      raise Exception(f"Sold units ({round(tot_units_sold)}) != target ({round(row_s['Units'])})")

    if VERBOSE:
      print("Updated Balance:")
      print(df_balance)
      print('-----------------------------------------------------------------------')
  else:
    raise Exception(f"Too many units to be sold! {row_s['Units']} > {df_balance_temp['Units'].sum()}")

# Add sales additional infos
df_sales = df_sales.join(pd.DataFrame(sales_infos))

df_sales

Original Balance:
         Date  Price   Units
0  2023-09-01  78.85   3.152
1  2023-10-02  77.04   3.225
2  2023-11-03  75.60  13.207
3  2023-12-02  79.79  12.514
4  2024-01-02  82.15  12.154
5  2024-02-01  84.99  11.749
6  2024-03-01  88.59  11.271
7  2024-04-02  90.63  11.017
8  2024-05-02  88.87  11.236
9  2024-06-03  91.82  10.875
10 2024-07-01  94.32  10.586
11 2024-08-01  95.27  10.481
12 2024-09-01  95.55  10.450
13 2024-09-15  94.70  10.544
14 2024-10-15  99.76  10.009 

Average Purchase Price: 88.3943859119827
Sold 100 units for a value of 12000. The 100 units where purchased at averaged price of 88.39, which means their average value is 8839.44
Gross Profit: 3160.56 (12000 - 8839.44)
Taxes: 869.15

Updated Balance:
         Date  Price     Units
0  2023-11-03  75.60  6.005231
1  2023-12-02  79.79  5.312231
2  2024-01-02  82.15  4.952231
3  2024-02-01  84.99  4.547231
4  2024-03-01  88.59  4.069231
5  2024-04-02  90.63  3.815231
6  2024-05-02  88.87  4.034231
7  2024-06-03  91

,Date,Units,Price,Value,Gross Profit,Taxes,Net Profit,Average Purchase Price
0,2024-10-16,100,120,12000,3160.56,869.15,2291.41,88.39
1,2024-10-17,52,100,5200,629.55,173.13,456.42,87.89
